# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_dataframe = pd.read_csv('./WeatherPy/weather_df')
weather_dataframe.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ngunguru,-35.62,174.50,59.00,3,38,1.99,NZ,1595197012
1,São João da Barra,-21.64,-41.05,71.60,78,0,11.41,BR,1595197013
2,The Valley,18.22,-63.06,86.00,66,40,13.87,AI,1595196999
3,Half Moon Bay,37.46,-122.43,75.00,77,90,6.93,US,1595196839
4,Khatanga,71.97,102.50,51.62,78,97,4.14,RU,1595197013


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [6]:
# storing lat and long into locations
locations = weather_dataframe[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store, be sure to handle NaN values
weather_dataframe = weather_dataframe.dropna()
humid = weather_dataframe["Humidity"].astype(float)

In [7]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid,
                                dissipating=False, max_intensity=100,
                                point_radius=1.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# create criteria for ideal weather conditions
max_temp = weather_dataframe.loc[weather_dataframe["Max Temp"] < 80, :]
min_temp = max_temp.loc[weather_dataframe["Max Temp"] > 70, :]
beaut_wind = min_temp.loc[min_temp["Wind Speed"]<10,:] 
beaut_clouds = beaut_wind.loc[beaut_wind["Cloudiness"] == 0, :]
beaut_cities = beaut_clouds
beaut_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Erenhot,43.65,111.98,70.09,34,0,5.59,CN,1595197014
75,Mizdah,31.45,12.98,71.55,64,0,9.78,LY,1595197023
81,Kāmyārān,34.80,46.94,73.40,12,0,4.70,IR,1595197023
120,Weinan,34.50,109.51,73.00,100,0,3.22,CN,1595197029
135,Cherskiy,68.75,161.30,70.34,46,0,7.29,RU,1595197031
144,Sirte,31.21,16.59,76.77,76,0,9.55,LY,1595197033
214,Abu Samrah,35.30,37.18,74.07,41,0,3.87,SY,1595197042
216,Três Passos,-27.46,-53.93,71.01,87,0,7.25,BR,1595197042
323,Xudat,41.63,48.68,79.16,63,0,3.58,AZ,1595197056
328,Jardim,-21.48,-56.14,76.08,42,0,4.32,BR,1595197057


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# store variable named hotel_df
hotel_df = beaut_cities.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the df
hotel_df["Hotel Name"] = ""



# find the hotels within 5000 meters
# set parameters"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# use iterrows to iterate through df
for index, row in hotel_df.iterrows():
    
    # get hotel type from df
    latitude = row["Lat"]
    longitude = row["Lng"]
    city = row["City"]
    country = row["Country"]
    
    # add keyword to params dict
    params["keyword"] = f"{latitude}, {longitude}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
                
    print("------------")


Retrieving Results for Index 9: Erenhot.
Missing field/result... skipping.
------------
Retrieving Results for Index 75: Mizdah.
Missing field/result... skipping.
------------
Retrieving Results for Index 81: Kāmyārān.
Missing field/result... skipping.
------------
Retrieving Results for Index 120: Weinan.
Missing field/result... skipping.
------------
Retrieving Results for Index 135: Cherskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 144: Sirte.
Missing field/result... skipping.
------------
Retrieving Results for Index 214: Abu Samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 216: Três Passos.
Missing field/result... skipping.
------------
Retrieving Results for Index 323: Xudat.
Missing field/result... skipping.
------------
Retrieving Results for Index 328: Jardim.
Missing field/result... skipping.
------------
Retrieving Results for Index 409: Shubarshi.
Missing field/result... skipping.
------------
Retrieving

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
